<a href="https://colab.research.google.com/github/KuroiYoshiko/Bullethole-detection/blob/main/Praca_Dyplomowa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Uczenie modelu**

### **Zamontowanie Dysku Google**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Ustawienie ścieżek oraz utworzenie folderów**

In [ ]:
import os

MY_MODEL_NAME = 'my_faster_rcnn_inception_resnet_v2_1024x1024'
PRETRAINED_MODEL_NAME = 'faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.tar.gz'
RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',MY_MODEL_NAME),
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',MY_MODEL_NAME, 'export'),
    'TFJSEXPORT_PATH':os.path.join('Tensorflow', 'workspace','models',MY_MODEL_NAME, 'tfjsexport'),
    'TFLITEEXPORT_PATH':os.path.join('Tensorflow', 'workspace','models',MY_MODEL_NAME, 'tfliteexport'),
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', MY_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], RECORD_SCRIPT_NAME),
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

### **Pobranie modelu z repozytorium Tensorflow Model Zoo i instalacja Tenserflow Object Detection**

In [ ]:
if os.name=='nt':
    !pip install wget
    import wget

if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

if os.name=='posix':
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install .

if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e .

Skrypt weryfikujący instalacje

In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
!python {VERIFICATION_SCRIPT}

Pobranie i wypakowanie modelu

In [ ]:
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

### **Import obrazów z Dysku Google**

Kopiowanie archiwum z obrazami z dysku google

In [ ]:
if os.name =='posix':
    !cp 'drive/MyDrive/Praca_Dyplomowa/archive.tar.gz' 'Tensorflow/workspace/images'
if os.name == 'nt':
    !copy 'drive/MyDrive/Praca_Dyplomowa/archive.tar.gz' 'Tensorflow/workspace/images'

Wypakowanie folderu z obrazami

In [ ]:
ARCHIVE_FILES = os.path.join(paths['IMAGE_PATH'], 'archive.tar.gz')
if os.path.exists(ARCHIVE_FILES):
  !tar -zxvf {ARCHIVE_FILES}

Tensorflow/workspace/images/train/
Tensorflow/workspace/images/train/WIN_20211121_12_42_49_Pro.jpg
Tensorflow/workspace/images/train/WIN_20211121_12_42_49_Pro.xml
Tensorflow/workspace/images/train/WIN_20211121_12_43_31_Pro.jpg
Tensorflow/workspace/images/train/WIN_20211121_12_43_31_Pro.xml
Tensorflow/workspace/images/train/WIN_20211121_12_44_03_Pro.jpg
Tensorflow/workspace/images/train/WIN_20211121_12_44_03_Pro.xml
Tensorflow/workspace/images/train/WIN_20211121_12_44_31_Pro.jpg
Tensorflow/workspace/images/train/WIN_20211121_12_44_31_Pro.xml
Tensorflow/workspace/images/train/WIN_20211121_12_50_24_Pro.jpg
Tensorflow/workspace/images/train/WIN_20211121_12_50_24_Pro.xml
Tensorflow/workspace/images/train/WIN_20211121_12_50_35_Pro.jpg
Tensorflow/workspace/images/train/WIN_20211121_12_50_35_Pro.xml
Tensorflow/workspace/images/train/WIN_20211121_12_51_24_Pro.jpg
Tensorflow/workspace/images/train/WIN_20211121_12_51_24_Pro.xml
Tensorflow/workspace/images/train/WIN_20211121_12_51_35_Pro.jpg
Tenso

### **Utworzenie rekordów treningowych i testowych**

Utworzenie Label Map

In [ ]:
labels = [{'name':'bullethole', 'id':1}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

Pobranie skryptu generującego pliki .record

In [ ]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

Tworzenie rekordów

In [ ]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')}
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')}

2022-02-09 19:51:55.429450: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
2022-02-09 19:51:59.957381: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


### **Zmiana parametrów pliku konfiguracyjnego modelu**

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

Kopiowanie pliku konfiguracyjnego modelu do folderu treningowego

In [ ]:
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

Zmiana parametrów

In [ ]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [ ]:
pipeline_config.model.faster_rcnn.num_classes = 1
pipeline_config.train_config.batch_size = 1
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [ ]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:
    f.write(config_text)

### **Trenowanie modelu**

In [ ]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=5000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

In [ ]:
!{command}

### **Ewaluacja modelu**

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

In [ ]:
!{command}

### **Import modelu z Dysku Google**

In [ ]:
if os.name =='posix':
    !cp 'drive/MyDrive/Praca_Dyplomowa/models.tar.gz' 'Tensorflow/workspace/models'
if os.name == 'nt':
    !copy 'drive/MyDrive/Praca_Dyplomowa/models.tar.gz' 'Tensorflow/workspace/models'

In [ ]:
MODEL_FILES = os.path.join(paths['MODEL_PATH'], 'models.tar.gz')
if os.path.exists(MODEL_FILES):
  !tar -zxvf {MODEL_FILES}

### **Detekcja obiektu z obrazu testowego**

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

Jeśli nie działa import cv2 konieczne jest wykonanie poniższych kroków:

In [ ]:
pip install opencv-python

In [ ]:
pip install opencv-python-headless==4.1.2.30

Detekcja:

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [ ]:
IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'test', 'WIN_20211121_13_20_36_Pro.jpg')
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.8,
            agnostic_mode=False)

plt.figure(figsize = (100,10))
plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()

### **Export modelu**

In [ ]:
!tar -czf models.tar.gz {paths['CHECKPOINT_PATH']}

# **Znalezienie współrzędnych dziur po kulach**

Wylistowanie wykrytych dziur

In [ ]:
boxes = detections['detection_boxes']
max_boxes_to_draw = boxes.shape[0]
scores = detections['detection_scores']
min_score_thresh=.8

bullethole_boxes = []

for i in range(min(max_boxes_to_draw, boxes.shape[0])):
    if scores is None or scores[i] > min_score_thresh:
        # (ymin, xmin, ymax, xmax)
        print("Bullethole", boxes[i])
        bullethole_boxes.append(boxes[i])

Bullethole [0.44297713 0.52673507 0.47095272 0.5440214 ]
Bullethole [0.36265644 0.43720692 0.38830653 0.45357057]
Bullethole [0.5122031  0.50233907 0.54074574 0.5189348 ]


Obliczenie środków odcinków boxów

In [ ]:
bulletholes = []

for i in range(len(bullethole_boxes)):
    # współrzędne narożników, bullethole_boxes = (ymin, xmin, ymax, xmax)
    a = [bullethole_boxes[i][1]*1920, bullethole_boxes[i][0]*1080]
    b = [bullethole_boxes[i][3]*1920, bullethole_boxes[i][0]*1080]
    c = [bullethole_boxes[i][3]*1920, bullethole_boxes[i][2]*1080]
    d = [bullethole_boxes[i][1]*1920, bullethole_boxes[i][2]*1080]

    # środki odcinków (styczne przestrzeleń)
    ab = [ round((a[0]+b[0])/2), round((a[1]+b[1])/2) ]
    bc = [ round((b[0]+c[0])/2), round((b[1]+c[1])/2) ]
    cd = [ round((c[0]+d[0])/2), round((c[1]+d[1])/2) ]
    da = [ round((d[0]+a[0])/2), round((d[1]+a[1])/2) ]

    bulletholes.append([ab, bc, cd, da])

# **Znalezienie współrzędnych środka tarczy i wszystkich okręgów punktowych**

Detekcja pierścieni punktowych i rysowanie ich na obrazach

In [ ]:
from google.colab.patches import cv2_imshow

img_copy = cv2.imread(IMAGE_PATH)
img_result = image_np_with_detections.copy()
bilateral_filtered_image = cv2.bilateralFilter(img_copy, 7, 175, 175)
gray = cv2.cvtColor(bilateral_filtered_image,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,100,200,apertureSize = 5, L2gradient = True)

target_circles_list = []
target_circles_radius_list = []
target_center = 0

#zdefiniowanie przedziałów promieniów, w które mogą osiągać pierścienie punktowe
for j in range(8):
    if j == 0:
        minRad = 40
        maxRad = 60
    elif j==1:
        minRad = 110
        maxRad = 130
    elif j==2:
        minRad = 180
        maxRad = 200
    elif j==3:
        minRad = 240
        maxRad = 260
    elif j==4:
        minRad = 310
        maxRad = 330
    elif j==5:
        minRad = 380
        maxRad = 400
    elif j==6:
        minRad = 440
        maxRad = 460
    elif j==7:
        minRad = 500
        maxRad = 520

    circle = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, dp=1, minDist=2000, param1=100, param2=30, minRadius=minRad, maxRadius=maxRad)
    circle = np.uint16(np.around(circle))

    for i in circle[0,:]:
        # draw the outer circle
        target_circles_list.append(i)
        target_circles_radius_list.append(i[2])
        cv2.circle(img_copy,(i[0],i[1]),i[2],(0,255,0),2)
        if j == 0:
            # draw the center of the circle
            target_center = [i[0], i[1]]
            cv2.circle(img_result,(i[0],i[1]),i[2],(255,0,0),2)
            cv2.circle(img_result,(i[0],i[1]),2,(0,0,255),3)
            cv2.circle(img_copy,(i[0],i[1]),2,(0,0,255),3)
        if j == 7:
          cv2.circle(img_result,(i[0],i[1]),i[2],(255,0,0),2)



print('circles list', target_circles_list)
print('circles radiuses', target_circles_radius_list)
print('target center', target_center)
cv2_imshow(img_copy)

Obliczenie odległości stycznych przestrzeleń do środka tarcz

In [ ]:
import math

bullethole_distances = []

for i in range(len(bulletholes)):
    distances = []
    for j in range(len(bulletholes[i])):
        distance = math.sqrt( (bulletholes[i][j][0] - target_center[0])**2 + (bulletholes[i][j][1] - target_center[1])**2 )
        distance = round(distance)
        distances.append(distance)
        if j>5:
            cv2.line(img, (bulletholes[i][j][0], bulletholes[i][j][1]),(target_center[0], target_center[1]),(255,0,0), 5)

        print (distance)
    bullethole_distances.append(distances)

88
100
82
67
149
128
129
149
55
77
81
62


Definiowanie tablicy punktów kolejnych pierścieni

In [ ]:
target_points = [10, 9, 8, 7, 6 , 5, 4, 3, 0]

Obliczenie maksymalnej ilości punktów dla każdego z punktów stycznych przestrzeleń

In [ ]:
point = 0
max_point_list = []

for i in range(len(bullethole_distances)):
  print('bullethole', i+1)
  max_point = []
  for j in range(len(bullethole_distances[i])):
    print('bullethole_distances[i][j] =', bullethole_distances[i][j])
    for k in range(len(target_circles_radius_list)):
      if bullethole_distances[i][j] > target_circles_radius_list[k]:
        point = target_points[k+1]
    print('punkty:', point)
    max_point.append(point)
  max_point_list.append(max_point)
  print()

bullethole 1
bullethole_distances[i][j] = 88
punkty: 9
bullethole_distances[i][j] = 100
punkty: 9
bullethole_distances[i][j] = 82
punkty: 9
bullethole_distances[i][j] = 67
punkty: 9

bullethole 2
bullethole_distances[i][j] = 149
punkty: 8
bullethole_distances[i][j] = 128
punkty: 8
bullethole_distances[i][j] = 129
punkty: 8
bullethole_distances[i][j] = 149
punkty: 8

bullethole 3
bullethole_distances[i][j] = 55
punkty: 9
bullethole_distances[i][j] = 77
punkty: 9
bullethole_distances[i][j] = 81
punkty: 9
bullethole_distances[i][j] = 62
punkty: 9



Wybieranie jedynie największej osiągniętej przez przestrzelenie punktacji

In [ ]:
bullethole_points = []
for i in range(len(max_point_list)):
  bullethole_points.append(max(max_point_list[i]))


Rysowanie odpowiednich znaczników graficznych na końcowym obrazie

In [ ]:
for i in range(len(bullethole_points)):
  cv2.putText(img_result, str(bullethole_points[i]), (bulletholes[i][2][0]+20, bulletholes[i][2][1]+20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2, cv2.FONT_HERSHEY_SIMPLEX)

In [ ]:
str_result = 'Result: ' + str(sum(bullethole_points)) + ' points'
cv2.putText(img_result, str_result, (50, 1050), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 2, cv2.FONT_HERSHEY_SIMPLEX)

Wyświetlenie końcowego wyniku detekcji

In [ ]:
cv2_imshow(img_result)

# **DETEKCJA W JEDNYM KROKU**

In [ ]:
IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'test', 'WIN_20211121_13_26_11_Pro.jpg')
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.8,
            agnostic_mode=False)


boxes = detections['detection_boxes']
max_boxes_to_draw = boxes.shape[0]
scores = detections['detection_scores']
min_score_thresh=.8

bullethole_boxes = []

for i in range(min(max_boxes_to_draw, boxes.shape[0])):
    if scores is None or scores[i] > min_score_thresh:
        # (ymin, xmin, ymax, xmax)
        print("Bullethole", boxes[i])
        bullethole_boxes.append(boxes[i])

bulletholes = []

for i in range(len(bullethole_boxes)):
    # współrzędne narożników, bullethole_boxes = (ymin, xmin, ymax, xmax)
    a = [bullethole_boxes[i][1]*1920, bullethole_boxes[i][0]*1080]
    b = [bullethole_boxes[i][3]*1920, bullethole_boxes[i][0]*1080]
    c = [bullethole_boxes[i][3]*1920, bullethole_boxes[i][2]*1080]
    d = [bullethole_boxes[i][1]*1920, bullethole_boxes[i][2]*1080]

    # środki odcinków (styczne przestrzeleń)
    ab = [ round((a[0]+b[0])/2), round((a[1]+b[1])/2) ]
    bc = [ round((b[0]+c[0])/2), round((b[1]+c[1])/2) ]
    cd = [ round((c[0]+d[0])/2), round((c[1]+d[1])/2) ]
    da = [ round((d[0]+a[0])/2), round((d[1]+a[1])/2) ]

    bulletholes.append([ab, bc, cd, da])

from google.colab.patches import cv2_imshow

img_copy = cv2.imread(IMAGE_PATH)
img_result = image_np_with_detections.copy()
bilateral_filtered_image = cv2.bilateralFilter(img_copy, 7, 175, 175)
gray = cv2.cvtColor(bilateral_filtered_image,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,100,200,apertureSize = 5, L2gradient = True)

target_circles_list = []
target_circles_radius_list = []
target_center = 0

#zdefiniowanie przedziałów promieniów, w które mogą osiągać pierścienie punktowe
for j in range(8):
    if j == 0:
        minRad = 40
        maxRad = 60
    elif j==1:
        minRad = 110
        maxRad = 130
    elif j==2:
        minRad = 180
        maxRad = 200
    elif j==3:
        minRad = 240
        maxRad = 260
    elif j==4:
        minRad = 310
        maxRad = 330
    elif j==5:
        minRad = 380
        maxRad = 400
    elif j==6:
        minRad = 440
        maxRad = 460
    elif j==7:
        minRad = 500
        maxRad = 520

    circle = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, dp=1, minDist=2000, param1=100, param2=30, minRadius=minRad, maxRadius=maxRad)
    circle = np.uint16(np.around(circle))

    for i in circle[0,:]:
        # draw the outer circle
        target_circles_list.append(i)
        target_circles_radius_list.append(i[2])
        cv2.circle(img_copy,(i[0],i[1]),i[2],(0,255,0),2)
        if j == 0:
            # draw the center of the circle
            target_center = [i[0], i[1]]
            cv2.circle(img_result,(i[0],i[1]),i[2],(255,0,0),2)
            cv2.circle(img_result,(i[0],i[1]),2,(0,0,255),3)
            cv2.circle(img_copy,(i[0],i[1]),2,(0,0,255),3)
        if j == 7:
          cv2.circle(img_result,(i[0],i[1]),i[2],(255,0,0),2)



print('circles list', target_circles_list)
print('circles radiuses', target_circles_radius_list)
print('target center', target_center)


import math

bullethole_distances = []

for i in range(len(bulletholes)):
    distances = []
    for j in range(len(bulletholes[i])):
        distance = math.sqrt( (bulletholes[i][j][0] - target_center[0])**2 + (bulletholes[i][j][1] - target_center[1])**2 )
        distance = round(distance)
        distances.append(distance)
        if j>5:
            cv2.line(img, (bulletholes[i][j][0], bulletholes[i][j][1]),(target_center[0], target_center[1]),(255,0,0), 5)

        print (distance)
    bullethole_distances.append(distances)

target_points = [10, 9, 8, 7, 6 , 5, 4, 3, 0]

point = 0
max_point_list = []

for i in range(len(bullethole_distances)):
  print('bullethole', i+1)
  max_point = []
  for j in range(len(bullethole_distances[i])):
    print('bullethole_distances[i][j] =', bullethole_distances[i][j])
    for k in range(len(target_circles_radius_list)):
      if bullethole_distances[i][j] > target_circles_radius_list[k]:
        point = target_points[k+1]
    print('punkty:', point)
    max_point.append(point)
  max_point_list.append(max_point)
  print()

bullethole_points = []
for i in range(len(max_point_list)):
  bullethole_points.append(max(max_point_list[i]))

for i in range(len(bullethole_points)):
  cv2.putText(img_result, str(bullethole_points[i]), (bulletholes[i][2][0]+20, bulletholes[i][2][1]+20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2, cv2.FONT_HERSHEY_SIMPLEX)

str_result = 'Result: ' + str(sum(bullethole_points)) + ' points'
cv2.putText(img_result, str_result, (50, 1050), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 2, cv2.FONT_HERSHEY_SIMPLEX)

cv2_imshow(img_result)